In [14]:
# Dependencies 
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import config

In [2]:
## Heather's Code
# Import CSV
fleet_csv = "./Resources/Fleet Data.csv"
fleet_df = pd.read_csv(fleet_csv)

# Dropping unnecessary columns, set the index 
cleaned_fleet = fleet_df.drop(columns=["Aircraft Type","Future","Historic","Total","Orders","Unit Cost","Average Age"]).fillna(value=0)
cleaned_fleet_df = cleaned_fleet.set_index(["Parent Airline"]+["Airline"])

# Remove ',','$' and blank values
cleaned_fleet_df["Total Cost (Current)"] = cleaned_fleet_df["Total Cost (Current)"].str.replace(',', '')
cleaned_fleet_df["Total Cost (Current)"] = cleaned_fleet_df["Total Cost (Current)"].str.replace('$', '')
cleaned_fleet_df = cleaned_fleet_df.dropna()

# Set the objects to integer
cleaned_fleet_df["Total Cost (Current)"] = cleaned_fleet_df["Total Cost (Current)"].astype(str).astype(int)

# Aggregate Columns to Sum
agg_fleet_df = cleaned_fleet_df.groupby(["Parent Airline","Airline"]).agg({"Current":"sum",
                                                                           "Total Cost (Current)":"sum"})
agg_fleet_df

Current  \
Parent Airline          Airline                                
Aegean Airlines         Aegean Airlines                 47.0   
                        Olympic Air                     14.0   
Aeroflot                Aeroflot                       189.0   
                        Aurora Airlines                 20.0   
                        Pobeda                          12.0   
                        Rossiya Airlines                61.0   
Aerolineas Argentinas   Aerolineas Argentinas           54.0   
                        Austral Líneas Aéreas           26.0   
Air Algerie             Air Algerie                     56.0   
Air Arabia              Air Arabia                      38.0   
                        Air Arabia Egypt                 1.0   
                        Air Arabia Jordan                2.0   
                        Air Arabia Maroc                 5.0   
Air Astana              Air Astana                      31.0   
Air Berlin              Air Berlin                     130.0   
Air Canada              Air Canada                     166.0   
                        Air Canada Jetz                  3.0   
                        Air Canada Regional              1.0   
                        Air Canada Rouge                44.0   
Air China               Air China                      375.0   
                        Air China Cargo                 15.0   
                        Air China Inner Mongolia         3.0   
                        Air Macau (66.9%)               17.0   
                        Dalian Airlines (80%)            9.0   
                        Kunming Airlines (80%)          18.0   
                        Shandong Airlines (51%)        100.0   
                        Shenzhen Airlines              171.0   
Air Europa              Air Europa                      51.0   
Air France/KLM          Air France                     226.0   
                        HOP!                            88.0   
...                                                      ...   
Tianjin Airlines        GX Airlines                     13.0   
                        Tianjin Airlines                91.0   
Trans States Airlines   Compass Airlines                62.0   
                        GoJet Airlines                  53.0   
                        Trans States Airlines           55.0   
Tunisair                Tunisair                        30.0   
                        Tunisair Express                 4.0   
Turkish Airlines        Anadolujet (100%)               34.0   
                        Turkish Airlines               295.0   
UPS Airlines            UPS                            236.0   
United Airlines         United Airlines                730.0   
Utair                   UTair-Cargo                     17.0   
                        Utair                           65.0   
                        Vostok Aviation Company         23.0   
Vietnam Airlines        Cambodia Angkor Air              6.0   
                        Jetstar Pacific Airlines        13.0   
                        Vietnam Air Service Company      4.0   
                        Vietnam Airlines                83.0   
Virgin America          Virgin America                  63.0   
Virgin Atlantic Airways Virgin Atlantic Airways         39.0   
Virgin Australia        Tigerair Australia              17.0   
                        Virgin Australia               113.0   
                        Virgin Australia Regional       16.0   
                        Virgin Samoa                     1.0   
Volaris                 Volaris                         68.0   
WestJet                 WestJet                        118.0   
                        WestJet Encore                  33.0   
Wizz Air                Wizz Air                        74.0   
easyJet                 easyJet                        234.0   
                        easyJet - Switzerland           26.0   

                    

In [12]:
## Michael's code
# Import CSV
airline_csv = "./Resources/Largest Airlines 2015 with Origin or Destination in USA.csv"
airline_df = pd.read_csv(airline_csv)
airline_df.columns

# Clean up the columns
cleaned_airline = airline_df.drop(columns=[" FREIGHT "," MAIL "])
cleaned_airline_df = cleaned_airline.rename(columns={" PASSENGERS ":"Passengers",
                                                     " Passenger Miles ":"Passenger Miles",
                                                     "CARRIER_NAME":"Parent Airline"})
# Remove ','
cleaned_airline_df["Passengers"] = cleaned_airline_df["Passengers"].str.replace(',', '')
cleaned_airline_df["Passenger Miles"] = cleaned_airline_df["Passenger Miles"].str.replace(',', '')

# Set the objects to integer
cleaned_airline_df["Passengers"] = cleaned_airline_df["Passengers"].astype(str).astype(np.int64)
cleaned_airline_df["Passenger Miles"] = cleaned_airline_df["Passenger Miles"].astype(str).astype(np.int64)

cleaned_airline_df

,Passengers,Passenger Miles,Parent Airline
0,144637180,116627136731,Southwest Airlines Co. Total
1,137916298,186140690281,Delta Air Lines Inc. Total
2,118541411,166869071222,American Airlines Inc. Total
3,95028877,182508547016,United Air Lines Inc. Total
4,35130683,41726258863,JetBlue Airways Total
5,30078314,16965403368,SkyWest Airlines Inc. Total
6,28269616,32330747593,US Airways Inc. Total
7,26002874,12683188091,ExpressJet Airlines Inc. Total
8,22868140,30301282863,Alaska Airlines Inc. Total
9,17629033,17944023815,Spirit Air Lines Total


In [17]:
# Create an engine
connection_string = f"{config.user}:{config.pw}@localhost:5432/fleet_db"
engine = create_engine(f'postgresql://{connection_string}')

In [16]:
# Confirm tables
engine.table_names()

OperationalError: (psycopg2.OperationalError) could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?

(Background on this error at: http://sqlalche.me/e/e3q8)